In [150]:
class Stack:
    def __init__(self):
        self.__list = []
    def isEmpty(self):
        return self.__list == []
    def push(self, item):
        self.__list.append(item)
    def pop(self):
        if self.isEmpty():
            return None
        else:
            return self.__list.pop()
    def clear(self):
        self.__list.clear()
    def peek(self):
        if self.isEmpty():
            return None
        else:
            return self.__list[len(self.__list)-1]
    def size(self):
        return len(self.__list)
    def get(self):
        if self.isEmpty():
            return None
        else:
            return self.__list[-1]
    def getValues(self):
        return [self._getNodeRepresentation(node) for node in self.__list]
    def _getNodeRepresentation(self, node):
        # Helper to extract key values from BinaryTree nodes
        return f"Node(key={node.getKey()}, left={self._childKey(node.getLeftTree())}, right={self._childKey(node.getRightTree())})"

    def _childKey(self, child):
        return child.getKey() if child is not None else "None"

def buildParseTree(exp):
    tokens = exp.replace('(', ' ( ').replace(')', ' ) ').split()
    stack = Stack()
    tree = BinaryTree('?')
    stack.push(tree)
    currentTree = tree

    print("Initial Stack:")
    print(stack.getValues())  # Display initial stack

    for t in tokens:
        print(f"\nProcessing token: '{t}'")

        # RULE 1: If token is '(' add a new node as left child
        # and descend into that node
        if t == '(':
            currentTree.insertLeft('?')
            stack.push(currentTree)
            currentTree = currentTree.getLeftTree() 

        # RULE 2: If token is operator set key of current node
        # to that operator and add a new node as right child
        # and descend into that node
        elif t in ['+', '-', '*', '/', '**']:
            currentTree.setKey(t)
            currentTree.insertRight('?')
            stack.push(currentTree)
            currentTree = currentTree.getRightTree()

        # RULE 3: If token is number, set key of the current node
        # to that number and return to parent
        elif t not in ['+', '-', '*', '/', ')', '**'] :
            try:
                currentTree.setKey(float(t))
                parent = stack.pop()
                currentTree = parent
            except ValueError:
                raise ValueError(f"Invalid token: {t}")

        
        # RULE 4: If token is ')' go to parent of current node
        elif t == ')':
            currentTree = stack.pop()
        else:
            raise ValueError

        print("Current Stack:")
        print(stack.getValues())

    return tree


class BinaryTree:
    def __init__(self,key, leftTree = None, rightTree = None):
        self.key = key
        self.leftTree = leftTree
        self.rightTree = rightTree
        self.myStack = []

    def setKey(self, key):
        self.key = key
    def getKey(self):
        return self.key
    def getLeftTree(self):
        return self.leftTree
    def getRightTree(self):
        return self.rightTree
    
    def insertLeft(self, key):
        if self.leftTree == None:
            self.leftTree = BinaryTree(key)
        else:
            t =BinaryTree(key)
            self.leftTree , t.leftTree = t, self.leftTree
    def insertRight(self, key):
        if self.rightTree == None:
            self.rightTree = BinaryTree(key)
        else:
            t =BinaryTree(key)
            self.rightTree , t.rightTree = t, self.rightTree

    def printPreorder(self, level):
        print( str(level*'-') + str(self.key))
        if self.leftTree != None:
            self.leftTree.printPreorder(level+1)
        if self.rightTree != None:
            self.rightTree.printPreorder(level+1) 

    def printInorder(self, level, leafStack):
        if self.leftTree != None:
            self.leftTree.printInorder(level+1, leafStack)
        print( str(level*'-') + str(self.key))
        leafStack.append([self.key, level])
        if self.rightTree != None:
            self.rightTree.printInorder(level+1, leafStack) 
        self.myStack = leafStack

    def printPostorder(self, level):
        if self.leftTree != None:
            self.leftTree.printPostorder(level+1)
        if self.rightTree != None:
            self.rightTree.printPostorder(level+1) 
        print( str(level*'-') + str(self.key))

def evaluate(tree):
    leftTree = tree.getLeftTree()
    rightTree = tree.getRightTree()
    op = tree.getKey()

    if leftTree != None and rightTree != None:
        if op == '+':
            return evaluate(leftTree) + evaluate(rightTree)
        elif op == '-':
            return evaluate(leftTree) - evaluate(rightTree)
        elif op == '*':
            return evaluate(leftTree) * evaluate(rightTree)
        elif op == '/':
            return evaluate(leftTree) / evaluate(rightTree)
        elif op == '**':
            return evaluate(leftTree) ** evaluate(rightTree)
    else:
        return (tree.getKey())

In [151]:
class Node:
    def __init__(self, data):
        self.data = data
        self.nextNode = None

    def __lt__(self, other):
        if self.data[1] != other.data[1]:
            return self.data[1] > other.data[1]  
        return len(self.data[0]) < len(other.data[0])  

    def __str__(self):
        return str(self.data)

class SortedList:
    def __init__(self):
        self.headNode = None
        self.currentNode = None
        self.length = 0

    def __appendToHead(self, newNode):
        oldHeadNode = self.headNode
        self.headNode = newNode
        self.headNode.nextNode = oldHeadNode

    def insert(self, newNode):
        # If list is currently empty
        if self.headNode == None:
            self.headNode = newNode
            return
    
        # Check if it is going to be new head
        if newNode < self.headNode:
            self.__appendToHead(newNode)
            return
        
        # Traverse and insert at appropriate location
        node = self.headNode

        while node.nextNode != None and not (newNode < node.nextNode):
            node = node.nextNode
        
        newNode.nextNode = node.nextNode
        node.nextNode = newNode
        
    
    def __str__(self):
        group = {}
        node = self.headNode
        while node != None:
            value = node.data[1]
            if value not in group:
                group[value] = []
            group[value].append(node.data[0])
            node = node.nextNode

        # Output
        output = []
        for key in sorted(group.keys(), reverse=True):
            output.append(f"*** Expressions with value=> {key}")
            for value in group[key]:
                output.append(f"{value}==>{key}")
            output.append("\n")

        return "\n".join(output)


In [152]:
def evaluateExpressions(expressions):
    results = []
    for exp in expressions:
        try:
            # Build parse tree
            tree = buildParseTree(exp)
            
            # Evaluate the parse tree
            result = evaluate(tree)
            
            # Append the expression and result as a tuple
            results.append((exp, result))
        except Exception as e:
            # Handle invalid expressions
            results.append((exp, f"Error: {e}"))
    return results


In [153]:
f = open("test1.txt", "r")
test = f.read()

print(test)

(106 - 100)
(10 + (20 + 30))
(2 + (2 + 2))
((11.07 + 25.5) - 10)
(((1 + 2 ) + 3 ) * 4)
((1 +2 )+( 3 + 4))
((10 + (10 + (10 + 10)))+(10 + 10))
((1 + 2 ) + (3 + ( 3 + 1 )))
((-500 + (4 * 3.14))/(2 ** 3))


(((((((((( 1 + 1 ) + 1 ) + 1 ) + 1 ) + 1 ) + 1 ) + 1 ) + 1 ) + 1 )) did not work so i remove it first

In [154]:
# I want you to split the expression into an array based on the spaces
# and then print the array
splitTest = test.splitlines()
evaluatedResults = evaluateExpressions(splitTest)


Initial Stack:
['Node(key=?, left=None, right=None)']

Processing token: '('
Current Stack:
['Node(key=?, left=?, right=None)', 'Node(key=?, left=?, right=None)']

Processing token: '106'
Current Stack:
['Node(key=?, left=106.0, right=None)']

Processing token: '-'
Current Stack:
['Node(key=-, left=106.0, right=?)', 'Node(key=-, left=106.0, right=?)']

Processing token: '100'
Current Stack:
['Node(key=-, left=106.0, right=100.0)']

Processing token: ')'
Current Stack:
[]
Initial Stack:
['Node(key=?, left=None, right=None)']

Processing token: '('
Current Stack:
['Node(key=?, left=?, right=None)', 'Node(key=?, left=?, right=None)']

Processing token: '10'
Current Stack:
['Node(key=?, left=10.0, right=None)']

Processing token: '+'
Current Stack:
['Node(key=+, left=10.0, right=?)', 'Node(key=+, left=10.0, right=?)']

Processing token: '('
Current Stack:
['Node(key=+, left=10.0, right=?)', 'Node(key=+, left=10.0, right=?)', 'Node(key=?, left=?, right=None)']

Processing token: '20'
Curren

In [155]:
for expr, result in evaluatedResults:
    print(f"{expr} = {result}")


(106 - 100) = 6.0
(10 + (20 + 30)) = 60.0
(2 + (2 + 2)) = 6.0
((11.07 + 25.5) - 10) = 26.57
(((1 + 2 ) + 3 ) * 4) = 24.0
((1 +2 )+( 3 + 4)) = 9.0
((10 + (10 + (10 + 10)))+(10 + 10)) = 60.0
((1 + 2 ) + (3 + ( 3 + 1 ))) = 10.0
((-500 + (4 * 3.14))/(2 ** 3)) = -60.93


In [156]:
evaluatedResults

[('(106 - 100)', 6.0),
 ('(10 + (20 + 30))', 60.0),
 ('(2 + (2 + 2))', 6.0),
 ('((11.07 + 25.5) - 10)', 26.57),
 ('(((1 + 2 ) + 3 ) * 4)', 24.0),
 ('((1 +2 )+( 3 + 4))', 9.0),
 ('((10 + (10 + (10 + 10)))+(10 + 10))', 60.0),
 ('((1 + 2 ) + (3 + ( 3 + 1 )))', 10.0),
 ('((-500 + (4 * 3.14))/(2 ** 3))', -60.93)]

In [157]:
sortedList = SortedList()


In [158]:
for expr, result in evaluatedResults:
    node = Node((expr, result))  # Create a Node with (expression, result)
    sortedList.insert(node)


print(sortedList)

*** Expressions with value=> 60.0
(10 + (20 + 30))==>60.0
((10 + (10 + (10 + 10)))+(10 + 10))==>60.0


*** Expressions with value=> 26.57
((11.07 + 25.5) - 10)==>26.57


*** Expressions with value=> 24.0
(((1 + 2 ) + 3 ) * 4)==>24.0


*** Expressions with value=> 10.0
((1 + 2 ) + (3 + ( 3 + 1 )))==>10.0


*** Expressions with value=> 9.0
((1 +2 )+( 3 + 4))==>9.0


*** Expressions with value=> 6.0
(106 - 100)==>6.0
(2 + (2 + 2))==>6.0


*** Expressions with value=> -60.93
((-500 + (4 * 3.14))/(2 ** 3))==>-60.93


